<a href="https://colab.research.google.com/github/gsdev42/Medical-question-summarization-/blob/main/metrics_for_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install torch transformers pillow clip
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.4 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-6s0dpobf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-6s0dpobf
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [1]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict
df = pd.read_csv('/content/drive/MyDrive/updated_clipsyntel_data.csv')
df.head()

,Question,Question_summ,image_path,category,context,description
0,11YR old daughter diagnosed MRSA+ 13 days ago ...,"What could be the cause of high fever, swollen...",swollen tonsils_Image_2.jpg,swollen tonsils,"[""Swollen tonsils, also known as tonsillitis, ...",Swollen tonsils appear enlarged and inflamed i...
1,"Hi, My 7 yr old son has very and they're very...",Should a parent worry about a 7-year-old with ...,swollen tonsils_Image_2.jpg,swollen tonsils,"[""Swollen tonsils, also known as tonsillitis, ...",Swollen tonsils appear enlarged and inflamed i...
2,"My 3 1/2 yo has red , high fever 103+ unmedica...",What other potential causes should be consider...,swollen tonsils_Image_9.jpg,swollen tonsils,"[""Swollen tonsils, also known as tonsillitis, ...",Swollen tonsils appear enlarged and inflamed i...
3,My 9 yr old daughter says she feels something ...,"Does a 9-year-old with swollen tonsils, feelin...",swollen tonsils_Image_2.jpg,swollen tonsils,"[""Swollen tonsils, also known as tonsillitis, ...",Swollen tonsils appear enlarged and inflamed i...
4,My son is 13 and for the last 2 or 3 years con...,Summary: Should a 13-year-old boy with recurre...,swollen tonsils_Image_3.jpg,swollen tonsils,"[""Swollen tonsils, also known as tonsillitis, ...",Swollen tonsils appear enlarged and inflamed i...


In [ ]:
#input column=> description, image_path
#output column=> category

In [2]:
!pip install scikit-learn

In [3]:
import torch
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# List of disorders
disorders = [
    'lip swelling', 'mouth ulcer', 'swollen tonsil', 'swollen eye', 'eye redness',
    'itchy eyelid', 'edema', 'foot swelling', 'knee swelling', 'hand lumps',
    'neck swelling', 'skin rash', 'skin irritation', 'skin growth', 'dry scalp',
    'cyanosis', 'eye inflammation', 'skin dryness'
]



# Initialize CLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to calculate similarity using CLIP
def clip_similarity(image_path, text):
    image_path = '/content/drive/MyDrive/Multimodal_img/'+ image_path
    image = processor(images=Image.open(image_path).convert("RGB"), return_tensors="pt")['pixel_values'].to(device)
    text = processor(text=[text], return_tensors="pt", padding=True)['input_ids'].to(device)

    with torch.no_grad():
        image_features = model.get_image_features(image)
        text_features = model.get_text_features(text)

    return torch.nn.functional.cosine_similarity(image_features, text_features).item()

# Function to predict top disorders
def predict_top_disorders(image_path, description):
    disorder_similarities = [(d, clip_similarity(image_path, d)) for d in disorders]
    top_disorders = sorted(disorder_similarities, key=lambda x: x[1], reverse=True)[:3]
    return [disorder for disorder, _ in top_disorders]

# Apply prediction and evaluate accuracy
correct_predictions = 0
total_samples = len(df)

for idx, row in df.iterrows():
    predicted_disorders = predict_top_disorders(row['image_path'], row['description'])
    print(f"Row {idx}: Predicted disorders - {predicted_disorders}")  # Debugging print

    if row['category'] in predicted_disorders:
        correct_predictions += 1



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Row 0: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 1: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 2: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 3: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 4: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 5: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 6: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 7: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 8: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 9: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 10: Predicted disorders - ['neck swelling', 'lip swelling', 'skin rash']
Row 11: Predicted disorders - ['mouth ulcer', 'swollen tonsil', 'lip swelling']
Row 12: Predicted disorders - ['mouth ulcer', 'swolle

In [4]:
accuracy = correct_predictions / total_samples * 100
print(f"Accuracy of the CLIP model: {accuracy:.2f}%")


Accuracy of the CLIP model: 75.59%


In [ ]:
from sklearn.metrics import f1_score ,recall_score , precision_score

# Initialize lists to store true labels and predictions
true_labels = []
predicted_labels_list = []

# Apply prediction and collect labels for F1 score calculation
for idx, row in df.iterrows():
    predicted_disorders = predict_top_disorders(row['image_path'], row['description'])
    true_labels.append(row['category'])
    # Append 1 if the true category is in the top 3 predicted categories, else 0
    predicted_labels_list.append(1 if row['category'] in predicted_disorders else 0)
    print(idx)  # Debugging print

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213


In [ ]:


# Since the F1 score function expects binary or multiclass classification outputs,
# We need to adjust true_labels and predicted_labels_list accordingly.
# Here we assume a binary classification (1 for correct, 0 for incorrect predictions).

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels_list, average='binary')
print(f"F1 Score of the CLIP model: {f1:.2f}")

# Calculate recall
recall = recall_score(true_labels, predicted_labels_list, average='binary')
print(f"Recall of the CLIP model: {recall:.2f}")

# Calculate precision
precision = precision_score(true_labels, predicted_labels_list, average='binary')
print(f"Precision of the CLIP model: {precision:.2f}")
